# Laboratorio 2 — Clasificación: detección de eventos raros (dataset sintético)

Aprenderás a diseñar un pipeline de clasificación y a evaluar correctamente cuando hay desbalance de clases.

## Objetivos
- Generar o cargar datos y entender el desbalance.
- Entrenar un baseline lineal y un modelo de ensamble.
- Usar métricas adecuadas (F1, matriz de confusión, ROC-AUC) y umbrales.

## Requisitos
- Laboratorio 0
- scikit-learn


## Contexto

Muchos problemas de IA en industria son *detección de eventos raros*: fraude, fallos, intrusiones, etc.
En estos escenarios **Accuracy** puede engañar: si el 98% es clase negativa, un modelo que siempre diga “no” ya logra 98%.


## 1) Crear un dataset con desbalance

Usaremos `make_classification` para simular datos tabulares sin depender de descargas externas.


In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=6000,
    n_features=20,
    n_informative=6,
    n_redundant=2,
    n_clusters_per_class=2,
    weights=[0.97, 0.03],  # 3% positivos
    flip_y=0.01,
    random_state=42
)

X = pd.DataFrame(X, columns=[f"f{i:02d}" for i in range(X.shape[1])])
y = pd.Series(y, name="evento_raro")

y.value_counts(normalize=True)


## 2) Split + baseline

Empezamos con regresión logística (con escalado).


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipe_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=500, class_weight="balanced"))
])

pipe_lr.fit(X_train, y_train)


## 3) Métricas que importan

Además de F1, observaremos:
- Matriz de confusión
- Curva ROC y AUC
- Precisión/Recall

La pregunta clave: **¿prefieres falsos positivos o falsos negativos?** Depende del negocio.


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, RocCurveDisplay
import matplotlib.pyplot as plt

y_pred = pipe_lr.predict(X_test)
y_proba = pipe_lr.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred, digits=3))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

RocCurveDisplay.from_predictions(y_test, y_proba)
plt.show()


## 4) Modelo alternativo: Gradient Boosting

Los modelos de boosting suelen rendir muy bien en tabular. Aquí probamos `HistGradientBoostingClassifier`.


In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

gb = HistGradientBoostingClassifier(
    max_depth=6,
    learning_rate=0.08,
    random_state=42
)
gb.fit(X_train, y_train)

y_pred_gb = gb.predict(X_test)
y_proba_gb = gb.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred_gb, digits=3))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_gb))


## 5) Ajuste de umbral

Por defecto, muchos clasificadores usan umbral 0.5. Con clases desbalanceadas, conviene seleccionar el umbral por una métrica objetivo (p.ej., maximizar F1 o asegurar recall mínimo).


In [ ]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

thresholds = np.linspace(0.05, 0.95, 19)

rows = []
for t in thresholds:
    pred_t = (y_proba_gb >= t).astype(int)
    rows.append({
        "umbral": t,
        "precision": precision_score(y_test, pred_t, zero_division=0),
        "recall": recall_score(y_test, pred_t),
        "f1": f1_score(y_test, pred_t, zero_division=0),
    })

df = pd.DataFrame(rows).sort_values("f1", ascending=False)
df.head(10)


## Ejercicios

### Ejercicio 1: Métrica orientada a negocio
Define un coste para FP y FN (por ejemplo, FP=1, FN=10). Encuentra el umbral que minimiza el coste esperado.

**Entregables**
- Función de coste
- Tabla coste vs umbral
- Umbral óptimo y justificación

**Criterios de evaluación**
- El coste se calcula correctamente
- Se justifica el umbral con números
- Se discute el trade-off

### Ejercicio 2: Comparación de modelos
Añade un tercer modelo (p.ej., `RandomForestClassifier` o `SVC`) y compáralo con LR y GB usando las mismas métricas.

**Entregables**
- Tabla comparativa
- Conclusión

**Pistas**
- En desbalance, prueba `class_weight='balanced'` o muestreo si procede

**Criterios de evaluación**
- Comparación justa (mismo split)
- Métricas relevantes
- Conclusión argumentada
